# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 8:26AM,249481,16,AIC-381152,American International Chemical,Released
1,Oct 27 2022 8:23AM,249480,22,631897,"NBTY Global, Inc.",Released
2,Oct 27 2022 8:21AM,249479,16,8696383,Sartorius Bioprocess Solutions,Released
3,Oct 27 2022 8:20AM,249478,16,8692914,Sartorius Bioprocess Solutions,Released
4,Oct 27 2022 8:15AM,249467,20,8696398-D,Reliable 1 Laboratories LLC,Released
5,Oct 27 2022 8:07AM,249476,10,Enova-10885,Emerginnova,Released
6,Oct 27 2022 8:07AM,249476,10,Enova-10886,Emerginnova,Released
7,Oct 27 2022 8:07AM,249476,10,Enova-10887,Emerginnova,Released
8,Oct 27 2022 8:07AM,249476,10,Enova-10888,Emerginnova,Released
9,Oct 27 2022 8:07AM,249476,10,DNMD-25600,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249476,Released,6
18,249478,Released,1
19,249479,Released,1
20,249480,Released,1
21,249481,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
249476,6
249478,1
249479,1
249480,1
249481,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
248349,1
248599,1
249407,1
249418,1
249447,1


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
248349,1
248599,1
249407,1
249418,1
249447,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,248349,1
1,248599,1
2,249407,1
3,249418,1
4,249447,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,248349,1
1,248599,1
2,249407,1
3,249418,1
4,249447,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 8:26AM,249481,16,American International Chemical
1,Oct 27 2022 8:23AM,249480,22,"NBTY Global, Inc."
2,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions
3,Oct 27 2022 8:20AM,249478,16,Sartorius Bioprocess Solutions
4,Oct 27 2022 8:15AM,249467,20,Reliable 1 Laboratories LLC
5,Oct 27 2022 8:07AM,249476,10,Emerginnova
11,Oct 27 2022 8:03AM,249475,19,"Methapharm, Inc."
17,Oct 27 2022 8:01AM,249474,19,"Methapharm, Inc."
20,Oct 27 2022 7:43AM,249473,19,"AdvaGen Pharma, Ltd"
45,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Oct 27 2022 8:26AM,249481,16,American International Chemical,1
1,Oct 27 2022 8:23AM,249480,22,"NBTY Global, Inc.",1
2,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,1
3,Oct 27 2022 8:20AM,249478,16,Sartorius Bioprocess Solutions,1
4,Oct 27 2022 8:15AM,249467,20,Reliable 1 Laboratories LLC,1
5,Oct 27 2022 8:07AM,249476,10,Emerginnova,6
6,Oct 27 2022 8:03AM,249475,19,"Methapharm, Inc.",6
7,Oct 27 2022 8:01AM,249474,19,"Methapharm, Inc.",3
8,Oct 27 2022 7:43AM,249473,19,"AdvaGen Pharma, Ltd",25
9,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Oct 27 2022 8:26AM,249481,16,American International Chemical,1
1,Oct 27 2022 8:23AM,249480,22,"NBTY Global, Inc.",1
2,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,1
3,Oct 27 2022 8:20AM,249478,16,Sartorius Bioprocess Solutions,1
4,Oct 27 2022 8:15AM,249467,20,Reliable 1 Laboratories LLC,1
5,Oct 27 2022 8:07AM,249476,10,Emerginnova,6
6,Oct 27 2022 8:03AM,249475,19,"Methapharm, Inc.",6
7,Oct 27 2022 8:01AM,249474,19,"Methapharm, Inc.",3
8,Oct 27 2022 7:43AM,249473,19,"AdvaGen Pharma, Ltd",25
9,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Oct 27 2022 8:26AM,249481,16,American International Chemical,1
1,Oct 27 2022 8:23AM,249480,22,"NBTY Global, Inc.",1
2,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,1
3,Oct 27 2022 8:20AM,249478,16,Sartorius Bioprocess Solutions,1
4,Oct 27 2022 8:15AM,249467,20,Reliable 1 Laboratories LLC,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Oct 27 2022 8:26AM,249481,16,American International Chemical,1
1,Oct 27 2022 8:23AM,249480,22,"NBTY Global, Inc.",1
2,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,1
3,Oct 27 2022 8:20AM,249478,16,Sartorius Bioprocess Solutions,1
4,Oct 27 2022 8:15AM,249467,20,Reliable 1 Laboratories LLC,1


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Oct 27 2022 8:26AM,249481,16,American International Chemical,1
1,Oct 27 2022 8:23AM,249480,22,"NBTY Global, Inc.",1
2,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,1
3,Oct 27 2022 8:20AM,249478,16,Sartorius Bioprocess Solutions,1
4,Oct 27 2022 8:15AM,249467,20,Reliable 1 Laboratories LLC,1


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,498938,11
12,498877,2
16,1247374,5
19,1247338,47
20,249467,1
21,995806,4
22,748401,3


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,498938,11
1,12,498877,2
2,16,1247374,5
3,19,1247338,47
4,20,249467,1
5,21,995806,4
6,22,748401,3


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,11
1,12,2
2,16,5
3,19,47
4,20,1
5,21,4
6,22,3


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,11
1,12,Released,2
2,16,Released,5
3,19,Released,47
4,20,Released,1


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Released,11,2,5,47,1,4,3


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Released,11,2,5,47,1,4,3


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Released,11,2,5,47,1,4,3


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()